In [ ]:
"""
Опис ходу думок і логіки виконання: 

1) на початку виконання завдання мені не вдалося отримати дані для тестування - і я вирішила відштовхуватися від тих двох зображень, що 
містилися в тексті завдання. Найбільш очевидним рішенням здавалося використати бібліотеки OpenCV і scikit-image.
"""

In [1]:
pip install opencv-python --no-binary opencv-python

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv-python-4.9.0.80.tar.gz (92.9 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for opencv-python: filename=opencv_python-4.9.0.80-cp36-cp36m-linux_x86_64.whl size=29880070 sha256=bcfe075482f88dcb88585d64026a863efc83270ed1df006e73506db116fa3e0d
  Stored in directory: /home/yuliya/.cache/pip/wheels/0c/97/47/bc0f95ddefcfd9c58c29a5148e2c2fee523d431d8821e72cc3
Successfully built opencv-python
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python==4.5.5.64

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 60.5 MB 11.2 MB/s            
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0.80
    Uninstalling opencv-python-4.9.0.80:
      Successfully uninstalled opencv-python-4.9.0.80
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python-headless==4.5.2.52

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 38.2 MB 5.0 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-image

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_image-0.17.2-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached imageio-2.15.0-py3-none-any.whl (3.3 MB)
  Using cached tifffile-2020.9.3-py3-none-any.whl (148 kB)
  Using cached PyWavelets-1.1.1-cp36-cp36m-manylinux1_x86_64.whl (4.4 MB)
  Using cached networkx-2.5.1-py3-none-any.whl (1.6 MB)
  Using cached Pillow-8.4.0-cp36-cp36m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
Note: you may need to rest

In [ ]:
"""
1.1) для порівняння зображень використала два методи з OpenCV - structural similarity i brute force matcher. Попередньо зображення переводилися в ч/б спектр і 
робилися більш контрастними. Для structural similarity також додатково уніфікувався розмір зображень.
В результаті, метод structural similarity показав неадекватний результат (0.04, що свідчить про несхожість порівнюваних зображень), а brute force matcher виявив 
співпадіння зображень (1.0)
"""

In [10]:
import cv2
from skimage.metrics import structural_similarity as ssim

def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply histogram equalization to enhance contrast
    equalized = cv2.equalizeHist(gray)
    return equalized

def compare_images_structurally(first_image_path, second_image_path):
    # Read images
    first_img = cv2.imread(first_image_path)
    second_img = cv2.imread(second_image_path)

    # Convert images to grayscale
    first_img_processed = preprocess_image(first_img)
    second_img_processed = preprocess_image(second_img)
    
    # Resize image (in case of different sizes)
    height, width = first_img_processed.shape
    print(first_img_processed.shape)
    height, width = second_img_processed.shape
    print(second_img_processed.shape)
    new_shape = (min(first_img_processed.shape[0], second_img_processed.shape[0]), min(first_img_processed.shape[1], second_img_processed.shape[1]))
    print(new_shape)
    first_img_resized = cv2.resize(first_img_processed,new_shape)
    second_img_resized = cv2.resize(second_img_processed,new_shape)

    # Compute SSIM between preprocessed images
    structural_similarity = ssim(first_img_resized, second_img_resized)
    print(f"Structural similarity is equal to {structural_similarity}")

    return structural_similarity

def extract_features(image):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def compare_images_with_brute_force_matcher(first_image_path, second_image_path):
    # Read images
    first_img = cv2.imread(first_image_path)
    second_img = cv2.imread(second_image_path)

    # Convert images to grayscale
    first_img_processed = preprocess_image(first_img)
    second_img_processed = preprocess_image(second_img)

    # Extract features from images
    kp1, des1 = extract_features(first_img_processed)
    kp2, des2 = extract_features(second_img_processed)

    # Initialize feature matcher
    bf = cv2.BFMatcher()
    # Match descriptors
    matches = bf.match(des1, des2)

    # Compute similarity score based on the number of matches
    match_similarity = len(matches) / max(len(kp1), len(kp2))
    print(f"Match similarity is equal to {match_similarity}")

    return match_similarity

image1_path = "/home/yuliya/Downloads/1_2.jpg"  
image2_path = "/home/yuliya/Downloads/2_2.jpg"  

# Compare images
match_score_ssim = compare_images_structurally(image1_path, image2_path)
match_score_bf = compare_images_with_brute_force_matcher(image1_path, image2_path)

(286, 383)
(288, 386)
(286, 383)
Structural similarity is equal to 0.04073243188322471
Match similarity is equal to 1.0


In [ ]:
"""
2) нарешті вдалося отримати набори тестових даних - і тут з'ясувалося, що OpenCV не тільки довго обробляє, але в принципі не в змозі коректно проаналізувати великі 
зображення - ані за допомогою brute force matcher(0.33), ані із structural similarity (0.2).
"""

In [1]:
import cv2
from skimage.metrics import structural_similarity as ssim

def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply histogram equalization to enhance contrast
    equalized = cv2.equalizeHist(gray)
    return equalized

def extract_features(image):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()
    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def compare_images_with_brute_force_matcher(first_image_path, second_image_path):
    # Read images
    first_img = cv2.imread(first_image_path)
    second_img = cv2.imread(second_image_path)
    print(3)
    # Convert images to grayscale
    first_img_processed = preprocess_image(first_img)
    second_img_processed = preprocess_image(second_img)
    print(4)
    # Extract features from images
    kp1, des1 = extract_features(first_img_processed)
    kp2, des2 = extract_features(second_img_processed)
    print(5)
    # Initialize feature matcher
    bf = cv2.BFMatcher()
    # Match descriptors
    matches = bf.match(des1, des2)
    print(6)
    # Compute similarity score based on the number of matches
    match_similarity = len(matches) / max(len(kp1), len(kp2))
    print(f"Match similarity is equal to {match_similarity}")

    return match_similarity
print(1)
image1_path = "/media/yuliya/Elements/LSMap4/RGB/sat_images/DJI_0267.JPG"  
image2_path = "/media/yuliya/Elements/LSMap4/RGB/uav_images/DJI_0267.JPG"  
print(2)
# Compare images
match_score_bf = compare_images_with_brute_force_matcher(image1_path, image2_path)

1
2
3
4
5
6
Match similarity is equal to 0.33337169099763053


In [2]:
import cv2
from skimage.metrics import structural_similarity as ssim

def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply histogram equalization to enhance contrast
    equalized = cv2.equalizeHist(gray)
    return equalized

def compare_images_structurally(first_image_path, second_image_path):
    # Read images
    first_img = cv2.imread(first_image_path)
    second_img = cv2.imread(second_image_path)

    # Convert images to grayscale
    first_img_processed = preprocess_image(first_img)
    second_img_processed = preprocess_image(second_img)
    
    # Resize image (in case of different sizes)
    height, width = first_img_processed.shape
    print(first_img_processed.shape)
    height, width = second_img_processed.shape
    print(second_img_processed.shape)
    new_shape = (min(first_img_processed.shape[0], second_img_processed.shape[0]), min(first_img_processed.shape[1], second_img_processed.shape[1]))
    print(new_shape)
    first_img_resized = cv2.resize(first_img_processed,new_shape)
    second_img_resized = cv2.resize(second_img_processed,new_shape)

    # Compute SSIM between preprocessed images
    structural_similarity = ssim(first_img_resized, second_img_resized)
    print(f"Structural similarity is equal to {structural_similarity}")

    return structural_similarity

image1_path = "/media/yuliya/Elements/LSMap4/RGB/sat_images/DJI_0267.JPG"  
image2_path = "/media/yuliya/Elements/LSMap4/RGB/uav_images/DJI_0267.JPG"  

# Compare images
match_score_ssim = compare_images_structurally(image1_path, image2_path)

(3730, 5472)
(3648, 5472)
(3648, 5472)
Structural similarity is equal to 0.20725055276211152


In [ ]:
"""
3) тоді вирішила обробити зображення за допомогою складніших фреймворків. Почала з PyTorch.
"""

In [3]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 519.9 kB/s eta 0:00:00m eta 0:00:0136m0:00:19m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 779.1 kB/s eta 0:00:00m eta 0:00:0136m0:00:08m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 992.3 kB/s eta 0:00:00m eta 0:00:016m0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 865.8 kB/s eta 0:00:00m eta 0:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 993.7 kB/s eta 0:00:00 eta 0:00:010:00:10m
DEPRECATION: distro-info 0.18ubuntu0.18.04.1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to 

In [5]:
pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 0.18ubuntu0.18.04.1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 882.1 kB/s eta 0:00:0031m1.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 1.3 MB/s eta 0:00:00m eta 0:00:010:00:01m
DEPRECATION: distro-info 0.18ubuntu0.18.04.1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psutil==5.9.1

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.6.
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 280 kB 1.4 MB/s            
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.7
    Uninstalling psutil-5.9.7:
      Successfully uninstalled psutil-5.9.7
Note: you may need to restart the kernel to use updated packages.


In [ ]:
"""
3.1) PyTorch показав непоганий результат схожості зображень (0.79) із використанням відтренованої моделі resnet50.
"""

In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def preprocess_image(image):
    # Preprocess the image for ResNet
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

def extract_features(image_path, model):
    # Read the image
    image = cv2.imread(image_path)
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Preprocess the image
    image_tensor = preprocess_image(image)
    # Extract features using the pretrained model
    with torch.no_grad():
        features = model(image_tensor)
    # Return the features
    return features.numpy().flatten()

def match_images(image1_path, image2_path, model):
    # Extract features from both images
    features1 = extract_features(image1_path, model)
    features2 = extract_features(image2_path, model)

    # Compute cosine similarity between the feature vectors
    similarity = cosine_similarity([features1], [features2])[0][0]

    return similarity
 
# Load the pretrained ResNet model
model = models.resnet50(pretrained=True)
# Set the model to evaluation mode
model.eval()

# Path to the images
image1_path = "/media/yuliya/Elements/LSMap4/RGB/sat_images/DJI_0267.JPG"  
image2_path = "/media/yuliya/Elements/LSMap4/RGB/uav_images/DJI_0267.JPG"

# Match the images
similarity_score = match_images(image1_path, image2_path, model)
print("Similarity score between the images:", similarity_score)

Similarity score between the images: 0.7894918


In [ ]:
"""
3.2) тоді я спробувала масштабувати рішення на повну виборку (400+ зображень), додавши асинхронну обробку (asyncio не зайшов, використала trio)
"""

In [7]:
pip install asyncio

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 245.1 kB/s eta 0:00:00 kB/s eta 0:00:01
DEPRECATION: distro-info 0.18ubuntu0.18.04.1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install trio

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 2.8 MB/s eta 0:00:002.6 MB/s eta 0:00:01
DEPRECATION: distro-info 0.18ubuntu0.18.04.1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [1]:
import trio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

def preprocess_image(image):
    # Preprocess the image for ResNet
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

def extract_features(image_path, model):
    # Read the image
    image = cv2.imread(image_path)
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Preprocess the image
    image_tensor = preprocess_image(image)
    # Extract features using the pretrained model
    with torch.no_grad():
        features = model(image_tensor)
    # Return the features
    return features.numpy().flatten()

async def match_images_async(image1_path, image2_path, model):
    similarity = await trio.to_thread.run_sync(match_images, image1_path, image2_path, model)
    return similarity

def match_images(image1_path, image2_path, model):
    # Extract features from both images
    features1 = extract_features(image1_path, model)
    features2 = extract_features(image2_path, model)

    # Compute cosine similarity between the feature vectors
    similarity = cosine_similarity([features1], [features2])[0][0]

    return similarity

async def process_image_pairs(folder_A, folder_B, model):
    async with trio.open_nursery() as nursery:
        for file_A, file_B in zip(os.listdir(folder_A), os.listdir(folder_B)):
            image1_path = os.path.join(folder_A, file_A)
            image2_path = os.path.join(folder_B, file_B)
            nursery.start_soon(match_and_print_similarity, image1_path, image2_path, model)

async def match_and_print_similarity(image1_path, image2_path, model):
    similarity = await match_images_async(image1_path, image2_path, model)
    file_A = os.path.basename(image1_path)
    file_B = os.path.basename(image2_path)
    print(f"Similarity score between {file_A} and {file_B}: {similarity}")

# Load the pretrained ResNet model
model = models.resnet50(pretrained=True)
# Set the model to evaluation mode
model.eval()

# Path to the folders containing the images
folder_A = "/media/yuliya/Elements/LSMap4/RGB/sat_images/"  
folder_B = "/media/yuliya/Elements/LSMap4/RGB/uav_images/"  

# Run the asynchronous processing of image pairs
trio.run(process_image_pairs, folder_A, folder_B, model)


/home/yuliya/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/yuliya/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Similarity score between DJI_0268.JPG and DJI_0268.JPG: 0.7443647980690002
Similarity score between DJI_0276.JPG and DJI_0276.JPG: 0.6568781733512878
Similarity score between DJI_0271.JPG and DJI_0271.JPG: 0.7963570356369019
Similarity score between DJI_0269.JPG and DJI_0269.JPG: 0.7925915122032166
Similarity score between DJI_0274.JPG and DJI_0274.JPG: 0.6340932250022888
Similarity score between DJI_0270.JPG and DJI_0270.JPG: 0.7926629185676575
Similarity score between DJI_0272.JPG and DJI_0272.JPG: 0.7322666645050049
Similarity score between DJI_0283.JPG and DJI_0283.JPG: 0.7614201307296753
Similarity score between DJI_0277.JPG and DJI_0277.JPG: 0.725383460521698
Similarity score between DJI_0267.JPG and DJI_0267.JPG: 0.7894917726516724
Similarity score between DJI_0289.JPG and DJI_0289.JPG: 0.739323616027832
Similarity score between DJI_0287.JPG and DJI_0287.JPG: 0.6671361327171326
Similarity score between DJI_0285.JPG and DJI_0285.JPG: 0.7553939819335938
Similarity score between DJ

In [ ]:
"""
3.3) додала оцінювання продуктивності. І звернула увагу, що є певний відсоток зображень, на яких similarity score менше 0.5 (напр., пари DJI_0523.JPG, 
DJI_0524.JPG, DJI_0687.JPG тощо). Перевірила кілька штук наочно - мають співпадати.
"""

In [23]:
import trio
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import cProfile
import pstats

def preprocess_image(image):
    # Preprocess the image for ResNet
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return preprocess(image).unsqueeze(0)

def extract_features(image_path, model):
    # Read the image
    image = cv2.imread(image_path)
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Preprocess the image
    image_tensor = preprocess_image(image)
    # Extract features using the pretrained model
    with torch.no_grad():
        features = model(image_tensor)
    # Return the features
    return features.numpy().flatten()

async def match_images_async(image1_path, image2_path, model):
    similarity = await trio.to_thread.run_sync(match_images, image1_path, image2_path, model)
    return similarity

def match_images(image1_path, image2_path, model):
    # Extract features from both images
    features1 = extract_features(image1_path, model)
    features2 = extract_features(image2_path, model)

    # Compute cosine similarity between the feature vectors
    similarity = cosine_similarity([features1], [features2])[0][0]

    return similarity

async def process_image_pairs(folder_A, folder_B, model):
    async with trio.open_nursery() as nursery:
        for file_A, file_B in zip(os.listdir(folder_A), os.listdir(folder_B)):
            image1_path = os.path.join(folder_A, file_A)
            image2_path = os.path.join(folder_B, file_B)
            nursery.start_soon(match_and_print_similarity, image1_path, image2_path, model)

async def match_and_print_similarity(image1_path, image2_path, model):
    similarity = await match_images_async(image1_path, image2_path, model)
    file_A = os.path.basename(image1_path)
    file_B = os.path.basename(image2_path)
    print(f"Similarity score between {file_A} and {file_B}: {similarity}")

# Load the pretrained ResNet model
model = models.resnet50(weights=models.resnet.ResNet50_Weights.IMAGENET1K_V1)
# Set the model to evaluation mode
model.eval()

# Path to the folders containing the images
folder_A = "/media/yuliya/Elements/LSMap4/RGB/sat_images/"  
folder_B = "/media/yuliya/Elements/LSMap4/RGB/uav_images/"  

# Perform profiling
profiler = cProfile.Profile()
profiler.enable()
trio.run(process_image_pairs, folder_A, folder_B, model)
profiler.disable()

# Print profiling results
stats = pstats.Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()


Similarity score between DJI_0667.JPG and DJI_0667.JPG: 0.4835648834705353
Similarity score between DJI_0689.JPG and DJI_0689.JPG: 0.5552326440811157
Similarity score between DJI_0688.JPG and DJI_0688.JPG: 0.5125524997711182
Similarity score between DJI_0690.JPG and DJI_0690.JPG: 0.6185286641120911
Similarity score between DJI_0691.JPG and DJI_0691.JPG: 0.5574139356613159
Similarity score between DJI_0669.JPG and DJI_0669.JPG: 0.5860558152198792
Similarity score between DJI_0686.JPG and DJI_0686.JPG: 0.5637882947921753
Similarity score between DJI_0685.JPG and DJI_0685.JPG: 0.5495849251747131
Similarity score between DJI_0668.JPG and DJI_0668.JPG: 0.5365264415740967
Similarity score between DJI_0675.JPG and DJI_0675.JPG: 0.5548352599143982
Similarity score between DJI_0682.JPG and DJI_0682.JPG: 0.5556017160415649
Similarity score between DJI_0663.JPG and DJI_0663.JPG: 0.5698270201683044
Similarity score between DJI_0683.JPG and DJI_0683.JPG: 0.5903065800666809
Similarity score between 

In [ ]:
"""
4) спробувала реалізувати все те саме - але із фреймворком TensorFlow і моделлю DenseNet201. Результат покращився (схожість зображень із показником менше 0.5 
не зустрічається), час виконання майже такий самий. 
"""

In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.0 MB/s eta 0:00:00m eta 0:00:010:01m:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 7.6 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:00m eta 0:00:010:01m:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 9.0 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 7.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.2 MB/s eta 0:00:00 MB/s eta 0:00:01m
   ━

In [2]:
import trio
import tensorflow as tf
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import cProfile
import pstats

# Load the DINO model
model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, pooling='avg')

# Define preprocessing function
def preprocess_image(image):
    # Preprocess the image for DINO
    image = cv2.resize(image, (224, 224))
    image = image[..., ::-1]  # BGR to RGB
    image = tf.keras.applications.densenet.preprocess_input(image)
    return image

# Define function for extracting features from an image
def extract_features(image):
    # Preprocess the image
    image = preprocess_image(image)
    # Add batch dimension
    image = tf.expand_dims(image, axis=0)
    # Extract features using the pretrained model
    features = model.predict(image)
    # Return the features
    return features.flatten()

# Define function for computing similarity between two images
def match_images(image1, image2):
    # Extract features from both images
    features1 = extract_features(image1)
    features2 = extract_features(image2)

    # Compute cosine similarity between the feature vectors
    similarity = cosine_similarity([features1], [features2])[0][0]

    return similarity

# Define function for processing image pairs asynchronously
async def process_image_pairs_async(folder_A, folder_B):
    files_A = os.listdir(folder_A)
    files_B = os.listdir(folder_B)

    # Iterate over image pairs
    for file_A, file_B in zip(files_A, files_B):
        image1 = cv2.imread(os.path.join(folder_A, file_A))
        image2 = cv2.imread(os.path.join(folder_B, file_B))
        # Run image pair processing asynchronously
        similarity = await trio.to_thread.run_sync(match_images, image1, image2)
        print(f"Similarity score between {file_A} and {file_B}: {similarity}")

# Path to the folders containing the images
folder_A = "/media/yuliya/Elements/LSMap4/RGB/sat_images/"  
folder_B = "/media/yuliya/Elements/LSMap4/RGB/uav_images/"    

# Perform profiling
profiler = cProfile.Profile()
profiler.enable()
trio.run(process_image_pairs_async, folder_A, folder_B)
profiler.disable()

# Print profiling results
stats = pstats.Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()


1/1 [==============================] - 0s 184ms/step
Similarity score between DJI_0267.JPG and DJI_0267.JPG: 0.7937214374542236
1/1 [==============================] - 0s 182ms/step
Similarity score between DJI_0268.JPG and DJI_0268.JPG: 0.8054308891296387
1/1 [==============================] - 0s 169ms/step
Similarity score between DJI_0269.JPG and DJI_0269.JPG: 0.7585002183914185
1/1 [==============================] - 0s 171ms/step
Similarity score between DJI_0270.JPG and DJI_0270.JPG: 0.7105798721313477
1/1 [==============================] - 0s 183ms/step
Similarity score between DJI_0271.JPG and DJI_0271.JPG: 0.70363450050354
1/1 [==============================] - 0s 180ms/step
Similarity score between DJI_0272.JPG and DJI_0272.JPG: 0.6794319152832031
1/1 [==============================] - 0s 171ms/step
Similarity score between DJI_0273.JPG and DJI_0273.JPG: 0.7163004875183105
1/1 [==============================] - 0s 164ms/step
Similarity score between DJI_0274.JPG and DJI_0274.JP